In [1]:
import numpy as np
import cv2
import time

# distance from camera to object(object) measured
# centemeter
KNOW_DISTANCE = 10

# width of the object in real
# centimeter
KNOWN_WIDTH = 3.4

## some colors 
Green = (0,255,0)
Red = (0,0,255)
White = (255,255,255)
Black = (0,0,0)

## define fonts
fonts = cv2.FONT_HERSHEY_COMPLEX


              ## here we will find the focal length 
def Focal_length_finder(measured_distance, real_width, width_in_ref_image):
    
    focal_length = (width_in_ref_image * measured_distance) / real_width
    
    return focal_length


              ## distance estimator fuction
def Distance_finder(Focal_length, real_width_object , object_width_in_frame):

    distace = (real_width_object * Focal_length) / object_width_in_frame

    return distace



def object_data(image):

    object_width = 0
    
        # convert the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)
    
        # find the contours in the edged image and keep the largest one;                          
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)  
    
        # we'll assume that this is our object in the image 
        ## this function use for sorting the values of the contour in a serial manner so taht we can use those values
        ## for further
    cnts = imutils.grab_contours(cnts)

        # A contour of an image is a curve joining all the continuous points along the boundary, 
        # having the same color or intensity. Contours are used for shape analysis and object detection and recognition etc.
        # To compute the area and perimeter of an object, we first detect the contour of the object and then apply cv2.    
    c = max(cnts, key = cv2.contourArea)

        # compute the bounding box of the of the object region and return it as 4_cornors of rectangle
        # (center(x, y), (width, height), angle of rotation) = cv2.minAreaRect(points)
    rec = cv2.minAreaRect(c)
    
        ## box contain all 4_lines dimensions (which means the 4 lines of a rectangle coordinates)
    box = cv2.cv.BoxPoints(rec) if imutils.is_cv2() else cv2.boxPoints(rec) 
            
        # np.int0() .... this helps to convert all the float type values inside the np.array to integer type
    box = np.int0(box)     

        ## draw line or connect all four points of the contours
    cv2.drawContours(image, [box], -1, Red, 2)
    
    
        ## we actually want to return the width of the object(pixel)
    return int(rec[1][0])



    ## reading reference image from directory 
reference_image = cv2.imread('photo1.jpeg')

    ## find the object width(pixels) in the reference_image
ref_image_object_width = object_data(reference_image)

    ## get the focal by calling "focal_length_finder"
    ## object width in reference(pixels)
    ## known distance in (centimeter)
    ## known width in (centimeter)
Focal_length_found = Focal_length_finder(KNOW_DISTANCE, KNOWN_WIDTH, ref_image_object_width)


    ## show the reference image 
cv2.imshow('window', reference_image)

    ## initialize the camera object so that we can get frame from it
cap = cv2.VideoCapture('video1.mp4')

    ## looping through frame, incoming from camera / or / video  
    ## while the camera or video is ON
while cap.isOpened():
    
        ## time dilay 0.5 milisecond
    time.sleep(0.05)

        ## reading the video 
      ## it will gives us 2 value...... (_) => True or False value .... if it detect the obj then it show True.. otherwise false
        ## (frame) => it capture photo from video..... so the photo data will be stored in frame
    _, frame  = cap.read()
    

    ## calling object_data fuction to find the width of the object(pixels) in the frame
    object_width_in_frame = object_data(frame)

    ## check if the object is zero then don't find the distance ... otherwise run the if loop 
    if object_width_in_frame != 0:

        # finding the distance by calling function 
        # Distance finder function need 
        # these arguments the Focal_Length,
        # Known_width(centimeters),
        # and Known_distance(centimeters)
        Distance = Distance_finder(Focal_length_found, KNOWN_WIDTH, object_width_in_frame)

        # print(Distance)
        
        ## draw line as background of text
        #cv2.line(frame, (30,30), (230,30), Red, 32)
        #cv2.line(frame, (30,30), (230,30), Black, 28)

        ## Put text on the screen
        cv2.putText(frame, f"Distance : {round(Distance ,2)} CM", (70,200), fonts, 0.6, Black, 2)

    ## show the frame on the screen
    cv2.imshow('window', frame)

    ## quit the program by pressing  "Q" button
    if cv2.waitKey(1) == ord("q"):
        break

## closing the camera
cap.release()

## closing the windows that are opened
cv2.destroyAllWindows()